### Autoencoders

Definición: https://www.ibm.com/es-es/topics/autoencoder 

Tipos:[ \[link\]](https://deeplearningofpython.blogspot.com/2023/05/Typesofautoencoders-implementation-keras.html?source=post_page-----383cfecc4d0e--------------------------------)

Como funcionan: https://www.geeksforgeeks.org/how-autoencoders-works/ 

- Compresión de imágenes y regeneración de imágenes
latent space: https://www.geeksforgeeks.org/what-is-latent-space-in-deep-learning/ 
Reconstrucción imágenes a partir del espacio latente https://medium.com/@jaiganesan/autoencoder-is-simple-9055a7b5b935 
Calidad de reconstrucción y pérdida de información: https://medium.com/@bhipanshudhupar/loss-functions-in-simple-autoencoders-mse-vs-l1-loss-4e838ae425b9 